In [1]:
#Load the data from previous phase;

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("DataPreprocessing") \
    .getOrCreate()

24/11/17 14:45:24 WARN Utils: Your hostname, Shwetas-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.1.101 instead (on interface en1)
24/11/17 14:45:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/17 14:45:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load traffic data
traffic_data = spark.read.csv("traffic", header=True, inferSchema=True)

# Load accident data
accident_data = spark.read.csv("accident", header=True, inferSchema=True)

In [5]:
traffic_data.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- hour_00: integer (nullable = true)
 |-- hour_01: integer (nullable = true)
 |-- hour_02: integer (nullable = true)
 |-- hour_03: integer (nullable = true)
 |-- hour_04: integer (nullable = true)
 |-- hour_05: integer (nullable = true)
 |-- hour_06: integer (nullable = true)
 |-- hour_07: integer (nullable = true)
 |-- hour_08: integer (nullable = true)
 |-- hour_09: integer (nullable = true)
 |-- hour_10: integer (nullable = true)
 |-- hour_11: integer (nullable = true)
 |-- hour_12: integer (nullable = true)
 |-- hour_13: integer (nullable = true)
 |-- hour_14: integer (nullable = true)
 |-- hour_15: integer (nullable = true)
 |-- hour_16: integer (nullable = true)
 |-- hour_17: integer (nullable = true)
 |-- hour_18: integer (nullable = true)
 |-- hour_19: integer (nullable = true)
 |-- hour_20: integer (nullable = true)
 |-- hour_21: integer (nullable = true)
 |-- hour_22: integer (nullable = true)
 |-- hour_23: integer (nullable =

In [6]:
accident_data.printSchema()

root
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- Visibility(mi): double (nullable = true)
 |-- Wind_Speed(mph): double (nullable = true)
 |-- Precipitation(in): double (nullable = true)
 |-- Amenity: integer (nullable = true)
 |-- Bump: integer (nullable = true)
 |-- Crossing: integer (nullable = true)
 |-- Give_Way: integer (nullable = true)
 |-- Junction: integer (nullable = true)
 |-- No_Exit: integer (nullable = true)
 |-- Railway: integer (nullable = true)
 |-- Roundabout: integer (nullable = true)
 |-- Station: integer (nullable = true)
 |-- Stop: integer (nullable = true)
 |-- Traffic_Calming: integer (null

In [7]:
# Rename columns in the accident_data DataFrame before joining
accident_data = accident_data.withColumnRenamed("year", "accident_year") \
                             .withColumnRenamed("month", "accident_month") \
                             .withColumnRenamed("day", "accident_day")

# Rename columns in the traffic_data DataFrame before joining
traffic_data = traffic_data.withColumnRenamed("year", "traffic_year") \
                           .withColumnRenamed("month", "traffic_month") \
                           .withColumnRenamed("day", "traffic_day")

In [8]:
#Everything is set now;

In [9]:
#The main objective here now is to join both the datasets, on date and exact time when accident occured and at what co-ordinates;

In [10]:
from pyspark.sql import functions as F

In [11]:
# Join accident_data with traffic_data on year, month, day, and nearest_station
data = accident_data.join(
    traffic_data,
    (accident_data["accident_year"] == traffic_data["traffic_year"]) &
    (accident_data["accident_month"] == traffic_data["traffic_month"]) &
    (accident_data["accident_day"] == traffic_data["traffic_day"]) &
    (accident_data["nearest_station"] == traffic_data["station_id"]),
    how="inner"
)

In [12]:
#Now we need to have a column which has traffic volume at the corresponding 'hour' from accident 'hour';|

In [13]:
# Create a traffic_volume column based on the hour from accident_data
traffic_volume_expr = F.when(F.col("hour") == 0, F.col("hour_00")) \
                      .when(F.col("hour") == 1, F.col("hour_01")) \
                      .when(F.col("hour") == 2, F.col("hour_02")) \
                      .when(F.col("hour") == 3, F.col("hour_03")) \
                      .when(F.col("hour") == 4, F.col("hour_04")) \
                      .when(F.col("hour") == 5, F.col("hour_05")) \
                      .when(F.col("hour") == 6, F.col("hour_06")) \
                      .when(F.col("hour") == 7, F.col("hour_07")) \
                      .when(F.col("hour") == 8, F.col("hour_08")) \
                      .when(F.col("hour") == 9, F.col("hour_09")) \
                      .when(F.col("hour") == 10, F.col("hour_10")) \
                      .when(F.col("hour") == 11, F.col("hour_11")) \
                      .when(F.col("hour") == 12, F.col("hour_12")) \
                      .when(F.col("hour") == 13, F.col("hour_13")) \
                      .when(F.col("hour") == 14, F.col("hour_14")) \
                      .when(F.col("hour") == 15, F.col("hour_15")) \
                      .when(F.col("hour") == 16, F.col("hour_16")) \
                      .when(F.col("hour") == 17, F.col("hour_17")) \
                      .when(F.col("hour") == 18, F.col("hour_18")) \
                      .when(F.col("hour") == 19, F.col("hour_19")) \
                      .when(F.col("hour") == 20, F.col("hour_20")) \
                      .when(F.col("hour") == 21, F.col("hour_21")) \
                      .when(F.col("hour") == 22, F.col("hour_22")) \
                      .when(F.col("hour") == 23, F.col("hour_23"))

In [14]:
# Add the traffic_volume column to data
data = data.withColumn("traffic_volume", traffic_volume_expr)

In [15]:
data.head(1)

24/11/17 14:45:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=25900Kb max_used=25971Kb free=105171Kb
 bounds [0x0000000105c6c000, 0x000000010760c000, 0x000000010dc6c000]
 total_blobs=10067 nmethods=9123 adapters=856
 compilation: disabled (not enough contiguous free space left)


[Row(Start_Lat=40.249505, Start_Lng=-75.66347900000002, Severity=0, Distance(mi)=0.021, City='Pottstown', State='PA', Temperature(F)=25.0, Wind_Chill(F)=25.0, Humidity(%)=85.0, Pressure(in)=30.14, Visibility(mi)=10.0, Wind_Speed(mph)=0.0, Precipitation(in)=0.0, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Duration_Minutes=80.17, accident_year=2021, accident_month=1, accident_day=1, hour=2, nearest_station='000011', distance_to_station_km=18.1, station_id='000011', hour_00=42, hour_01=48, hour_02=28, hour_03=27, hour_04=15, hour_05=22, hour_06=18, hour_07=17, hour_08=28, hour_09=35, hour_10=56, hour_11=75, hour_12=75, hour_13=98, hour_14=69, hour_15=68, hour_16=47, hour_17=66, hour_18=39, hour_19=49, hour_20=53, hour_21=43, hour_22=48, hour_23=36, traffic_year=2021, traffic_month=1, traffic_day=1, traffic_volume=28)]

In [16]:
#We can see the hour=2 is being compared to hour_02 which has traffic volume of 28; thus assigning this value to traffic_volume column

In [17]:
#We will now remove redundant columns

In [18]:
columns_to_drop = [
    "hour_00", "hour_01", "hour_02", "hour_03", "hour_04", "hour_05", 
    "hour_06", "hour_07", "hour_08", "hour_09", "hour_10", "hour_11", 
    "hour_12", "hour_13", "hour_14", "hour_15", "hour_16", "hour_17", 
    "hour_18", "hour_19", "hour_20", "hour_21", "hour_22", "hour_23", 
    "station_id", "traffic_year", "traffic_month", "traffic_day" 
]

In [19]:
data = data.drop(*columns_to_drop)

In [20]:
data.head(1)

[Row(Start_Lat=40.249505, Start_Lng=-75.66347900000002, Severity=0, Distance(mi)=0.021, City='Pottstown', State='PA', Temperature(F)=25.0, Wind_Chill(F)=25.0, Humidity(%)=85.0, Pressure(in)=30.14, Visibility(mi)=10.0, Wind_Speed(mph)=0.0, Precipitation(in)=0.0, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Duration_Minutes=80.17, accident_year=2021, accident_month=1, accident_day=1, hour=2, nearest_station='000011', distance_to_station_km=18.1, traffic_volume=28)]

In [21]:
#We will convert "accident_year=2021, accident_month=1, accident_day=1" back to proper date

In [22]:
data = data.withColumn(
    "date",
    F.to_date(
        F.concat_ws("-",
                    F.col("accident_year"),
                    F.lpad(F.col("accident_month").cast("string"), 2, "0"),  # Ensure two-digit month
                    F.lpad(F.col("accident_day").cast("string"), 2, "0")     # Ensure two-digit day
                   ),
        "yyyy-MM-dd"
    )
).drop("accident_year", "accident_month", "accident_day")

In [23]:
data.head(1)

[Row(Start_Lat=40.249505, Start_Lng=-75.66347900000002, Severity=0, Distance(mi)=0.021, City='Pottstown', State='PA', Temperature(F)=25.0, Wind_Chill(F)=25.0, Humidity(%)=85.0, Pressure(in)=30.14, Visibility(mi)=10.0, Wind_Speed(mph)=0.0, Precipitation(in)=0.0, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Duration_Minutes=80.17, hour=2, nearest_station='000011', distance_to_station_km=18.1, traffic_volume=28, date=datetime.date(2021, 1, 1))]

In [24]:
#Based on the columns we got, feature engineering will be helpful for improving performance

In [25]:
#Morning - 6am - 12pm

In [26]:
#Afternoon - 12pm - 6pm

In [27]:
#Evening - 6pm - 12am

In [28]:
#Night - 12am - 6am

In [29]:
# Categorize hour into broader categories (morning, afternoon, evening, night)
data = data.withColumn(
    "time_of_day",
    F.when((F.col("hour") >= 6) & (F.col("hour") < 12), "Morning")
     .when((F.col("hour") >= 12) & (F.col("hour") < 18), "Afternoon")
     .when((F.col("hour") >= 18) & (F.col("hour") < 24), "Evening")
     .otherwise("Night")
)

In [30]:
# Drop the hour column
data = data.drop("hour")

In [31]:
# Extract the day of the week from the date column
data = data.withColumn("day_of_week", F.date_format("date", "E"))  # 'E' gives day of the week (Mon, Tue, etc.)

In [32]:
# Create a binary feature indicating whether the day is a weekend
data = data.withColumn(
    "is_weekend",
    F.when(F.col("day_of_week").isin("Sat", "Sun"), 1).otherwise(0)
)

In [33]:
data.head(1)

[Row(Start_Lat=40.249505, Start_Lng=-75.66347900000002, Severity=0, Distance(mi)=0.021, City='Pottstown', State='PA', Temperature(F)=25.0, Wind_Chill(F)=25.0, Humidity(%)=85.0, Pressure(in)=30.14, Visibility(mi)=10.0, Wind_Speed(mph)=0.0, Precipitation(in)=0.0, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Duration_Minutes=80.17, nearest_station='000011', distance_to_station_km=18.1, traffic_volume=28, date=datetime.date(2021, 1, 1), time_of_day='Night', day_of_week='Fri', is_weekend=0)]

In [34]:
# Drop duplicate rows from the DataFrame
data = data.dropDuplicates()

In [35]:
#We will save the data for further processing

In [36]:
# Saving traffic_data DataFrame as CSV
data.coalesce(1).write.csv("data", header=True)

24/11/17 14:48:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/17 14:48:17 WARN RowBasedKeyValueBatch: Calling spill() on

In [37]:
#We have successfully inner joined both datasets and pre-processed accordingly

In [38]:
###########################################################################################################################################

In [39]:
###########################################################################################################################################